### Building a sigmoid neural network model to recognize digit number.
##### using batch gradient descent

In [864]:
import pandas as pd
import numpy as np
import random

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.metrics import accuracy_score

sigmoid gradient
$${g}'(z)=\frac{d}{dz}g(z)=g(z)(1-g(z))$$
where
$$sigmoid(z)=g(z)=\frac{1}{1+e^{-z}}$$

### Cost Function with regularization
$$J(\theta )) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K}[-y_{k}^{(i)}log((h_\theta(x^{(i)}))_{k}) - (1-y_{k}^{(i)})log(1-(h_{\theta}(x^{(i)}))_{k})] + \frac{\lambda }{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_{l}}\sum_{j=1}^{s_{l+1}}(\theta_{ji}^{(l)})^{2}$$

- Training set is $${(x^{1}, y^{1}), (x^{2}, y^{2}), (x^{3}, y^{3}) ... (x^{m}, y^{m})}$$
- L = number of layers in the network
- S<sub>l</sub> = number of units (not counting bias unit) in layer l 
- K is number of units in output layer S<sub>L</sub> = K
-  $\theta^{l}_{ji}$ is the weight from the ith unit of layer l to jth unit of layer l+1

In [879]:
## TODO complement explanation of neural network

In [4]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [5]:
def sigmoidGradient(z):
    return sigmoid(z)*(1 - sigmoid(z))

In [843]:
def initWeight(numUnits):
    w = []
    epsilon_init = 0.01
    for i in range(len(numUnits)-1):
        w.append(np.random.random((numUnits[i] + 1, numUnits[i+1]))*epsilon_init)
    return w

In [504]:
def NNcost(y, y_pre, theta, lamb):
    m = len(y)
    L = len(theta)
    regular = 0
    # calculate regularization term
    for i in range(L):
        # bias weight not included
        regular += np.sum(np.square(theta[i][:,1:])) 
    regular = regular * lamb / m
    cost = np.sum(- y * np.log(y_pre) - (1 - y) * np.log(1 - y_pre))/m + regular
    return cost

### train

In [505]:
def gradient(x, y, theta, lamb):
    # number of theta
    L = len(theta)
    # number of data sets
    m = len(x)
    # save every a and z for backpropagation
    a = []
    z = []
    current_a = x
    current_z = 0
    for i in range(L):
        # add bias to input
        current_a = np.c_[np.ones(m),current_a]
        a.append(current_a)
        current_z = np.dot(current_a , theta[i])
        z.append(current_z)
        current_a = sigmoid(current_z)
    
    # value of cost function
    cost = NNcost(y, current_a, theta, lamb)
    
    # calculate the gradient of theta for updating
    gradient = []
    error = []
    l = L-1
    current_error = current_a - y
    error.append(current_error)
    delta = []
    while l > 0:
        current_error = np.dot(current_error,np.transpose(theta[l])) * sigmoidGradient(np.c_[np.ones(len(z[l-1])),z[l-1]])
        current_error = current_error[:,1:]
        error.append(current_error)
        l -= 1
        
    for j in range(L):
        delta.append(np.dot(np.transpose(a[j]),error[L-1-j]))
    
    theta_temp = [np.c_[np.zeros(len(t)),t[:,1:]] for t in theta]
    theta_grad = []
    
    for i in range(L):
        theta_grad.append(delta[i]/m + lamb/m * theta_temp[i])
    
    return (cost, theta_grad)

In [874]:
def updateTheta(alpha, theta, theta_grad):
    # batch gradient descent
    for i in range(len(theta)):
        theta[i] = theta[i] - alpha*theta_grad[i]
    return theta

In [878]:
def train(x, y, theta, alpha, lamb, max_cost, max_iter):
    cost = 1
    k = 0
    while cost > max_cost and k < max_iter:
        cost,theta_grad = gradient(x,y,theta,lamb)
        theta = updateTheta(alpha, theta, theta_grad)
        k = k + 1
    return theta, cost

In [848]:
def predict(x, theta):
    # number of layers
    L = len(theta)
    # number of data sets
    m = len(x)
    a = x
    for i in range(L):
        # add bias to input
        a = np.c_[np.ones(m),a]
        z = np.dot(a , theta[i])
        a = sigmoid(z)
    return a

In [721]:
def oneHot(m):
    n = len(np.unique(m))
    r = np.zeros((len(m), n))
    for i in range(len(m)):
        r[i,m[i]] = 1
    return r

In [2]:
# data from https://www.kaggle.com/c/digit-recognizer
df_train=pd.read_csv('data/train.csv')

In [768]:
y_train = df_train["label"]

In [825]:
X_train = df_train.drop("label", axis=1)/256.0

In [770]:
# trains y_train to one hot format
y_train = oneHot(y_train)

In [872]:
# initialize theta
theta = initWeight([784, 40, 10])
alpha = 0.2
lamb = 1
max_cost = 0.01
max_iter = 10000

In [873]:
# train the model
theta, cost = train(X_train[:10000], y_train[:10000], theta, alpha, lamb, max_cost, max_iter)

In [875]:
# predict the result of test set
y_pre = predict(X_train[10000:12000], theta)

In [877]:
accuracy_score(np.round(y_pre), y_train[10000:12000])

0.90749999999999997